# [모듈 1.1] EventBridge를 통해 CodePipeline 수행

이 노트북은 CodeCommit 레파지토리에 'SageMaker Pipeline config 파일(sm_pipeline_train_config.json)'이 push 되면, EventBridge를 통해 CodePipeline을 실행하는 노트북 입니다.

## 참고 자료
- Boto3 Docs : EventBridge
   (https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/events.html)
- Boto3 Docs : IAM
   (https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iam.html)



# 1. 환경 확인

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import sagemaker
import boto3
sys.path.append('./src')

%store -r s3_input_data_uri
print("s3_input_data_uri: n", s3_input_data_uri)


# 2. 파라미터 세팅

In [ ]:
# 기본 정보
region = sagemaker.Session().boto_region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')
training_files = 'training'

# Codecommit Repo Name
repository_name = "ncf-train"

# code pipeline name, arn, role_arn
codepipeline_name = "ncf-training-code-pipeline"
codepipeline_arn = boto3.client('codepipeline').get_pipeline(name = codepipeline_name).get('metadata').get('pipelineArn')

# EventBridge Rule Name
eventbridge_rule_name = "codepipelinerule"

# EventBridge CodePipeline Execution Role and Policy
eventbridge_role_name = 'eventbridge_codepipeline_execution_role'
eventbridge_policy_name = 'eventbridge_invoke_codepipeline_policy'

# Training data (S3 prefix)
data_prefix = "NCFModel/data2"

# 3. EventBridge Rule 정의

## 3.1 EventPattern 정의

#### 아래 CodeCommit arn 출력값을 복사해서 resources 에 붙여넣으세요

In [ ]:
# Codecommit Arn
repository_arn = boto3.client('codecommit').get_repository(repositoryName = repository_name).get('repositoryMetadata').get('Arn')
print(repository_arn)

In [ ]:
eventclient = boto3.client('events')

eventresponse = eventclient.put_rule(
    Name = eventbridge_rule_name,
    EventPattern = """{
        "source": ["aws.codecommit"],
        "detail-type": ["CodeCommit Repository State Change"],
        "resources": ["arn:aws:codecommit:us-east-1:376278017302:ncf-train"],
        "detail": {
            "referenceType": ["branch"],
            "referenceName": ["master"]
        }
    }"""
)

print(eventresponse)

## 3.2 Event Target (CodePipeline excution)을 위한 IAM Role 생성

### 3.2.1 Role 생성

In [ ]:
import json
import boto3

# Create an IAM client
iam = boto3.client('iam')

trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "events.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

# Create a new IAM role with the necessary permissions
response2 = iam.create_role(
    RoleName = eventbridge_role_name,
    AssumeRolePolicyDocument = json.dumps(trust_policy)
)

# Get the Amazon Resource Name (ARN) of the newly-created role
role_arn = response2['Role']['Arn']
print(role_arn)

### 3.2.2 Policy 생성

In [ ]:
import json
import boto3

# Create an IAM client
iam = boto3.client('iam')

eventbridge_invoke_codepipeline_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "codepipeline:StartPipelineExecution",
            "Resource": "*"
        }
    ]
}

response1 = iam.create_policy(
    PolicyName = eventbridge_policy_name,
    Path = '/',
    PolicyDocument = json.dumps(eventbridge_invoke_codepipeline_policy)
)

# Get the Amazon Resource Name (ARN) of the newly-created policy
policy_arn = response1['Policy']['Arn']
print(policy_arn)

### 3.2.3 Role에 Policy Attach

In [ ]:
response3 = iam.attach_role_policy(
    RoleName = eventbridge_role_name,
    PolicyArn = policy_arn
)
print(response3)

## 3.3 Event Target 설정

In [ ]:
eventresponse = eventclient.put_targets(
    Rule = eventbridge_rule_name,
    Targets = [
        {
            'Arn' : codepipeline_arn,
            'Id' : codepipeline_name,
            'RoleArn' : role_arn
        }
    ]
)

print(eventresponse)

# 4. 리파지토리 로컬에 클로닝
### Repository URL을 확인 후 복사해서 아래의 repository_url에 붙여넣기 해주세요.
### repository_url :
```
repository_url =  "https://git-codecommit.us-east-1.amazonaws.com/v1/repos/ncf-train"
```

In [ ]:
repository_url =  "https://git-codecommit.us-east-1.amazonaws.com/v1/repos/ncf-train"
# repository_url = "<Repository URL>"

In [ ]:
%%sh -s {repository_url} {repository_name}

repository_url=$1
repository_name=$2

rm -rf $repository_name

git clone $repository_url 

# 5. 새로운 SageMaker Pipeline (Training) 구성 파일 준비

## 5.1. 변수 설정 및 dict 로 저장

In [ ]:
project_prefix = "SageMaker-Train-NCF"
bucket = sagemaker.Session().default_bucket()

sm_pipeline_name = "ncf-sm-pipeline"
ModelApprovalStatus="PendingManualApproval"
inference_image_uri = "763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.8.1-gpu-py3"
training_instance_type = "ml.p3.2xlarge"
training_instance_count = 1

sm_pipeline_train_config_json_path = f'src/sm_pipeline_train_config.json'

In [ ]:
sm_train_pipeline_dict = {
    "project_prefix" : project_prefix,            
    "s3_input_data_uri" : s3_input_data_uri,
    "sm_pipeline_name" : sm_pipeline_name,
    "training_instance_type" :  training_instance_type,    
    "training_instance_count" :  training_instance_count,        
    "ModelApprovalStatus" :  ModelApprovalStatus,    
    "inference_image_uri" :  inference_image_uri,
    "bucket" : bucket,            
}

## 5.2. 설정 딕션너리 Json 파일로 저장 후 테스트를 위해 로딩

In [ ]:
from common_utils import save_json, load_json

save_json(sm_pipeline_train_config_json_path, sm_train_pipeline_dict)
sm_pipeline_train_dict = load_json(sm_pipeline_train_config_json_path)
print (json.dumps(sm_pipeline_train_dict, indent=2))

## 5.3 Config 파일 복사

### 5.3.1 sm_pipeline_train_config_json 파일

In [ ]:
source_path = sm_pipeline_train_config_json_path
target_path = 'codecommit/pipelines/ncf/src/'

! cp {source_path} {target_path}

# 6. SageMaker Pipeline config 파일을 로컬 레파지토리 디렉토리에 복사

In [ ]:
%%sh -s {repository_name} {training_files}
repository_name=$1
training_files=$2

cp $training_files/* $repository_name
ls $repository_name

# 7. 리파지토리에 푸시 (이벤트 발생)

In [ ]:
%%sh -s {repository_name} 
repository_name=$1
cd $repository_name
echo $PWD
git add .
git commit -m "Add new files"
git push origin master


# 8. 변수 저장

In [ ]:
%store eventbridge_rule_name
%store codepipeline_name
%store eventbridge_role_name
%store eventbridge_policy_name
%store policy_arn
%store sm_pipeline_train_config_json_path